In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from matplotlib import pyplot
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
from scipy.stats import ttest_ind

from numpy import median

In [2]:
df = pd.read_csv('data/func_class_p0.05_effectsize2.0.csv')
df['gen'] = [True if r[:3] == 'sps' else False for r in df['sp_id'].values]
df = df[df['gen'] == True]
df = df[df['correct'] == True]
df = df[df['protein_id'] != 'Arnold_033']

In [3]:
gen_sps = list(set(df['sp_id'].values))

In [4]:
sp_df = pd.read_excel("data/sp_prot_translations.xls")

spid_to_seq = dict()
for sp in gen_sps:
    seqID = 'seq' + sp[3:-2]
    version = sp[-1]
    
    if version == '1':
        seq = sp_df[sp_df['seqID']==seqID].iloc[0]['75'][:-6]
    elif version == '2':
        seq = sp_df[sp_df['seqID']==seqID].iloc[0]['90'][:-6]
    elif version == '3':
        seq = sp_df[sp_df['seqID']==seqID].iloc[0]['95'][:-6]
    elif version == '4':
        seq = sp_df[sp_df['seqID']==seqID].iloc[0]['99'][:-6]
        
    spid_to_seq.update({sp:seq})

In [5]:
df

,Unnamed: 0,correct,protein_id,sp_id,run_label,seqID,prot_correct,sp_correct,family,run_id,run_index,assay,assay_value,old_correct,plot_id,func,gen
0,0,True,Arnold_002,sps12-2,amylase_1,seq19,True,True,amylase,19_12-2,21,assay_3_value,0.502,True,NaN,True,True
1,1,True,Arnold_002,sps12-2,amylase_1,seq19,True,True,amylase,19_12-2,33,assay_1_value,0.378,True,Pro_02-2,True,True
2,2,True,Arnold_002,sps12-2,amylase_1,seq19,True,True,amylase,19_12-2,21,assay_4_value,0.618,True,NaN,True,True
3,3,True,Arnold_002,sps12-2,amylase_1,seq19,True,True,amylase,19_12-2,21,assay_1_value,0.430,True,Pro_02-2,True,True
4,4,True,Arnold_002,sps12-2,amylase_1,seq19,True,True,amylase,19_12-2,33,assay_2_value,0.294,True,NaN,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1342,1342,True,Arnold_039,sps39-2,xylanase_1,seq3,True,True,xylanase,3_39-2,583,assay_4_value,0.994,False (-SP/+Protein),NaN,False,True
1343,1343,True,Arnold_039,sps39-2,xylanase_1,seq3,True,True,xylanase,3_39-2,583,assay_3_value,0.622,False (-SP/+Protein),NaN,False,True
1344,1344,True,Arnold_039,sps39-2,xylanase_1,seq3,True,True,xylanase,3_39-2,535,assay_3_value,0.612,True,NaN,False,True
1345,1345,True,Arnold_039,sps39-2,xylanase_1,seq3,True,True,xylanase,3_39-2,535,assay_5_value,1.938,True,NaN,False,True


In [6]:
df['sp_seq'] = df['sp_id'].map(spid_to_seq)

In [7]:
df = df[['family', 'protein_id', 'sp_id', 'func', 'sp_seq']]

In [8]:
df

,family,protein_id,sp_id,func,sp_seq
0,amylase,Arnold_002,sps12-2,True,MTSYEFLLVILGVLLSGA
1,amylase,Arnold_002,sps12-2,True,MTSYEFLLVILGVLLSGA
2,amylase,Arnold_002,sps12-2,True,MTSYEFLLVILGVLLSGA
3,amylase,Arnold_002,sps12-2,True,MTSYEFLLVILGVLLSGA
4,amylase,Arnold_002,sps12-2,True,MTSYEFLLVILGVLLSGA
...,...,...,...,...,...
1342,xylanase,Arnold_039,sps39-2,False,MNISIFVGKLALAALGSALVA
1343,xylanase,Arnold_039,sps39-2,False,MNISIFVGKLALAALGSALVA
1344,xylanase,Arnold_039,sps39-2,False,MNISIFVGKLALAALGSALVA
1345,xylanase,Arnold_039,sps39-2,False,MNISIFVGKLALAALGSALVA


In [9]:
func_df = df[df['func']==True]
nonfunc_df = df[df['func']==False]

func_sps = list(set(func_df['sp_seq'].values))
nonfunc_sps = list(set(nonfunc_df['sp_seq'].values))
all_sps = list(set(func_sps + nonfunc_sps))

In [10]:
onlyfunc_sps = [s for s in func_sps if s not in nonfunc_sps ]
r_onlyfunc_sps = [s for s in func_sps if s in nonfunc_sps ]
onlynonfunc_sps = [s for s in nonfunc_sps if s not in func_sps ]
r_onlynonfunc_sps = [s for s in nonfunc_sps if s in func_sps ]

print(len(set(func_sps)), len(set(onlyfunc_sps)), len(set(nonfunc_sps)), len(set(onlynonfunc_sps)))

def classify(row):
    if row['sp_seq'] in onlyfunc_sps:
        return "only_func"
    elif row['sp_seq'] in onlynonfunc_sps:
        return "only_nonfunc"
    else:
        return "both"

df['sp_gen_class'] = df.apply(lambda x: classify(x), axis=1)

43 27 46 30


<ipython-input-10-842bdd5830e9>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sp_gen_class'] = df.apply(lambda x: classify(x), axis=1)


In [11]:
df = df.drop_duplicates()
df['level'] = [s[-1] for s in df['sp_id']]
df

<ipython-input-11-66e14fd72682>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['level'] = [s[-1] for s in df['sp_id']]


,family,protein_id,sp_id,func,sp_seq,sp_gen_class,level
0,amylase,Arnold_002,sps12-2,True,MTSYEFLLVILGVLLSGA,both,2
8,amylase,Arnold_002,sps12-4,True,MNIRLGALLAGLLLSAMASAVFA,both,4
12,amylase,Arnold_002,sps19-1,True,MVSFKSALFAAAAVATVADA,both,1
16,amylase,Arnold_002,sps19-2,True,MQKKTAIAIAAGTAIATVAAGTQA,both,2
24,amylase,Arnold_002,sps19-3,True,MVSFSSLLAAASLAVVNA,both,3
...,...,...,...,...,...,...,...
1287,xylanase,Arnold_039,sps3-1,False,MRLIVFLATSATSLFASLA,both,1
1297,xylanase,Arnold_039,sps3-2,False,MFKLKDILIGLTGILLSSLFA,only_nonfunc,2
1307,xylanase,Arnold_039,sps3-4,False,MRLAKIAGLTASLLFSLWGALA,only_nonfunc,4
1327,xylanase,Arnold_039,sps39-1,False,MLSIDTSSTRRVVPNTALFPNTHRRDFATAGQLLAMASAVLTGAPAHA,only_nonfunc,1


In [12]:
df = df[df['sp_gen_class'] != 'both']

In [13]:
# Kyte & Doolittle index of hydrophobicity 
# J. Mol. Biol. 157:105-132(1982). 
kd = {"A": 1.8, "R": -4.5, "N": -3.5, "D": -3.5, "C": 2.5, 
    "Q": -3.5, "E": -3.5, "G": -0.4, "H": -3.2, "I": 4.5, 
    "L": 3.8, "K": -3.9, "M": 1.9, "F": 2.8, "P": -1.6, 
    "S": -0.8, "T": -0.7, "W": -0.9, "Y": -1.3, "V": 4.2} 

# Hydrophilicity 
# 1 Hopp & Wood 
# Proc. Natl. Acad. Sci. U.S.A. 78:3824-3828(1981). 
hw = {"A": -0.5, "R": 3.0, "N": 0.2, "D": 3.0, "C": -1.0, 
    "Q": 0.2, "E": 3.0, "G": 0.0, "H": -0.5, "I": -1.8, 
    "L": -1.8, "K": 3.0, "M": -1.3, "F": -2.5, "P": 0.0, 
    "S": 0.3, "T": -0.4, "W": -3.4, "Y": -2.3, "V": -1.5} 

In [14]:
# Calculate properties, put into df

from Bio import SeqIO
from Bio.SeqUtils import ProtParam

new_data = []

for seq in onlyfunc_sps + onlynonfunc_sps:
    X = ProtParam.ProteinAnalysis(seq)
    # Gen stats
    level = df[df['sp_seq']==seq].iloc[0]['level']
    func_class = df[df['sp_seq']==seq].iloc[0]['sp_gen_class']
    # Manual stats
    hydrophobicity = np.average([kd[aa] for aa in seq])
    hydrophilicity = np.average([hw[aa] for aa in seq])
    # New data
    new_data.append([seq, func_class, len(seq),
                     hydrophobicity, hydrophilicity, 
                     X.molecular_weight(), X.molecular_weight()/len(seq),
                     X.aromaticity(), X.instability_index(), X.isoelectric_point()])
cols = ['seq', 'class', 'length', 'hydrophob', 'hydrophil', 'MolWeight', 'avg MW', 'aromaticity', 'instability', 'isoelectric']

new_df = pd.DataFrame(new_data, columns=cols)
new_df.head()

,seq,class,length,hydrophob,hydrophil,MolWeight,avg MW,aromaticity,instability,isoelectric
0,MNKKFKTIMALAIATLSAAGVGVAHA,only_func,26,0.876923,-0.407692,2615.1653,100.583281,0.038462,11.157692,10.301331
1,MKKKIVAVLTLSVVLA,only_func,16,1.562500,-0.412500,1713.2196,107.076225,0.000000,-1.237500,10.301331
2,MKKKIAITLLFLSLLNRA,only_func,18,0.838889,-0.294444,2073.6305,115.201694,0.055556,4.727778,11.263733
3,MKKRVISALAALWLSVLGAPAVLA,only_func,24,1.333333,-0.558333,2479.0786,103.294942,0.041667,50.687500,11.165955
4,MKMRTGKKGFLSILLAFLLVITSIPFTLVDVEA,only_func,33,1.072727,-0.436364,3653.4840,110.711636,0.090909,15.818182,9.697083


In [15]:
from scipy.stats import ttest_ind

In [16]:
props = cols[2:]
print(props)

for prop in props:
    funcs = list(map(float, new_df[new_df['class'] == 'only_func'][prop].values))
    nons = list(map(float, new_df[new_df['class'] == 'only_nonfunc'][prop].values))
    p = ttest_ind(funcs, nons, equal_var=True).pvalue
    print(f"{prop} : \t {p:.4f}")

['length', 'hydrophob', 'hydrophil', 'MolWeight', 'avg MW', 'aromaticity', 'instability', 'isoelectric']
length : 	 0.9229
hydrophob : 	 0.3780
hydrophil : 	 0.2126
MolWeight : 	 0.7963
avg MW : 	 0.1170
aromaticity : 	 0.9335
instability : 	 0.9748
isoelectric : 	 0.3243
